# Technical Assessment

### The first task is to determine shot distribution of shots attempted by each team within each zone. The second task is to determine the effective field goal percentage of each team's shots within each zone. 

First, the provided data will be loaded into a Pandas DataFrame.

In [1]:
import pandas as pd
import numpy as np
from math import sqrt
shots = pd.read_csv('shots_data.csv') 
shots

,team,x,y,fgmade
0,Team A,-23.1,3.5,0
1,Team A,0.0,25.1,1
2,Team A,0.5,1.0,1
3,Team A,-5.6,5.2,0
4,Team A,4.0,14.5,1
...,...,...,...,...
555,Team B,7.8,24.7,0
556,Team B,10.1,1.4,1
557,Team B,-17.8,-0.1,0
558,Team B,22.6,2.9,0


Next, a function to determine the shot type of each shot will be created, and then a new column in the Pandas DataFrame called ShotType will be created.

In [2]:
def type_of_shot(x, y):
    shot_type = ""
    distance = sqrt((x**2) + (y**2))
    x = abs(x)
    if y <= 7.8:
        if x > 22:
            shot_type += "C3"
        else:
            shot_type += "2PT"
    else: 
        if distance > 23.75:
            shot_type += "NC3"
        else:
            shot_type += "2PT"
            
    return shot_type

In [3]:
shots["ShotType"] = shots.apply(lambda x: type_of_shot(x['x'], x['y']), axis=1)
shots

,team,x,y,fgmade,ShotType
0,Team A,-23.1,3.5,0,C3
1,Team A,0.0,25.1,1,NC3
2,Team A,0.5,1.0,1,2PT
3,Team A,-5.6,5.2,0,2PT
4,Team A,4.0,14.5,1,2PT
...,...,...,...,...,...
555,Team B,7.8,24.7,0,NC3
556,Team B,10.1,1.4,1,2PT
557,Team B,-17.8,-0.1,0,2PT
558,Team B,22.6,2.9,0,C3


### Now that the shot type of each shot has been determined, I can begin to construct a new dataframe with the information necessary for the task. 

First, the dataframe will be split into two smaller dataframes, each of which holds one teams data. The data will then be sorted by shot type to congregate the shots of similar type.

In [4]:
teamA_shots = shots.iloc[:280]
teamB_shots = shots.iloc[280:]
teamA_shots = teamA_shots.sort_values(["ShotType"])
teamB_shots = teamB_shots.sort_values(["ShotType"])

Now the functions for calculating shot distribution and effective field goal percentage can be created, as well as the helper function shots_taken.

In [5]:
def shots_taken(dataframe, shot_type):
    total = 0
    for index, row in dataframe.iterrows():
        if row['ShotType'] == shot_type:
            total += 1
    return total 

In [6]:
def shot_distribution(dataframe, shot_type):
    return str(round((shots_taken(dataframe, shot_type) / len(dataframe) * 100), 3)) + "%"

In [7]:
def eFG(dataframe, shot_type):
    attempted = shots_taken(dataframe, shot_type)
    made = 0
    for index, row in dataframe.iterrows():
        if row['ShotType'] == shot_type:
            made += row['fgmade']
    if shot_type != '2PT':
        made = made + (made * 0.5)
    eFG = made / attempted
    return str(round(eFG * 100, 3)) + "%"
    return made

In [8]:
def total_eFG(dataframe):
    attempted = len(dataframe)
    twos = 0
    threes = 0
    attempted = 0
    for index, row in dataframe.iterrows():
        if row['ShotType'] == '2PT':
            twos += row['fgmade']
            attempted += 1
        else:
            threes += row['fgmade']
            attempted += 1
    made = twos + threes
    threes = threes * 0.5
    top = made + threes
    eFG = top / attempted
    return str(round(eFG * 100, 3)) + "%"

Finally, the empty dataframe in which we will put all of our information will be developed and filled.

In [9]:
team_data = pd.DataFrame(index=['Team A', 'Team B'], columns=['2PT ShotDist', 'NC3 ShotDist', 'C3 ShotDist', '2PT eFG', 'NC3 eFG', 'C3 eFG', 'Total eFG'])

In [10]:
team_data.at["Team A", "2PT ShotDist"] = shot_distribution(teamA_shots, '2PT')
team_data.at["Team A", "NC3 ShotDist"] = shot_distribution(teamA_shots, 'NC3')
team_data.at["Team A", "C3 ShotDist"] = shot_distribution(teamA_shots, 'C3')
team_data.at["Team A", "2PT eFG"] = eFG(teamA_shots, '2PT')
team_data.at["Team A", "NC3 eFG"] = eFG(teamA_shots, 'NC3')
team_data.at["Team A", "C3 eFG"] = eFG(teamA_shots, 'C3')
team_data.at["Team A", "Total eFG"] = total_eFG(teamA_shots)
team_data.at["Team B", "2PT ShotDist"] = shot_distribution(teamB_shots, '2PT')
team_data.at["Team B", "NC3 ShotDist"] = shot_distribution(teamB_shots, 'NC3')
team_data.at["Team B", "C3 ShotDist"] = shot_distribution(teamB_shots, 'C3')
team_data.at["Team B", "2PT eFG"] = eFG(teamB_shots, '2PT')
team_data.at["Team B", "NC3 eFG"] = eFG(teamB_shots, 'NC3')
team_data.at["Team B", "C3 eFG"] = eFG(teamB_shots, 'C3')
team_data.at["Team B", "Total eFG"] = total_eFG(teamB_shots)

In [11]:
team_data

,2PT ShotDist,NC3 ShotDist,C3 ShotDist,2PT eFG,NC3 eFG,C3 eFG,Total eFG
Team A,60.714%,32.143%,7.143%,40.0%,48.333%,60.0%,44.107%
Team B,58.214%,34.286%,7.5%,46.012%,54.688%,35.714%,48.214%


The dataframe above includes the shot distribution for each team within each zone, Corner 3, Non-Corner 3, and 2 Point, as well as each teams effective field goal percentage in each zone and the total effective field goal percentage for each team overall. All of the data has been put in percentages and rounded to the nearest one-thousandth. 